In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT_CLASSES_ACCURACY
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from torchvision.transforms import Compose
from hmpai.pytorch.transforms import *
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from mne.io import read_info
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

In [2]:
set_global_seed(42)
data_path_sat2 = DATA_PATH / "sat2/stage_data_proba_100hz.nc"
dataset_sat2 = xr.load_dataset(data_path_sat2)
# data_path_sat1 = DATA_PATH / "sat1/stage_data_100hz.nc"
# dataset_sat1 = xr.load_dataset(data_path_sat1)
# dataset_sat1 = dataset_sat1.rename_vars({'RT': 'rt'})

# dataset_acc = dataset_sat1.where(dataset_sat1.event_name.str.contains("accuracy"), drop=True)
# dataset_sp = dataset_sat1.where(dataset_sat1.event_name.str.contains("speed"), drop=True)

In [3]:
# shape_topological = False
# info_to_keep = ['rt']
# train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
#     dataset_sat2, 60, norm_min1_to_1
# )
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
# train_dataset = SAT1Dataset(train_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# val_dataset = SAT1Dataset(val_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# test_dataset = SAT1Dataset(test_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)

In [4]:
shape_topological = False
info_to_keep = ["rt"]
train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
    dataset_sat2, 60, norm_min1_to_1
)
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
train_dataset = SAT1Dataset(
    train_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([ShuffleOperations()]),
)
val_dataset = SAT1Dataset(
    val_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)
test_dataset = SAT1Dataset(
    test_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)

In [5]:
model = Seq2SeqTransformer(d_model=30, ff_dim=2048, num_heads=8, num_layers=6, num_classes=len(SAT_CLASSES_ACCURACY), emb_dim=1024)
model.set_pretraining(False)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=8,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=10,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.1,
    lr=0.00001
)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/323 [00:00<?, ? batch/s]

  0%|          | 0/323 [00:00<?, ? batch/s]

  0%|          | 0/323 [00:00<?, ? batch/s]

  0%|          | 0/323 [00:00<?, ? batch/s]

...

  0%|          | 0/323 [00:00<?, ? batch/s]

  0%|          | 0/323 [00:00<?, ? batch/s]

  0%|          | 0/323 [00:00<?, ? batch/s]

  0%|          | 0/323 [00:00<?, ? batch/s]

ValueError: Found input variables with inconsistent numbers of samples: [6397210, 1279442]